### Задание 1

In [41]:
import requests, time, json
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def get_pages():
    
    """
    Функция выводит список страниц в формате:
    
        [https://habr.com/ru/all/page1/
        https://habr.com/ru/all/page2/
        ...]
    
    """
    URL = 'https://habr.com/ru/all/' 
    
    habr = requests.get(URL)
    time.sleep(0.2)
    soup = BeautifulSoup(habr.text, 'html.parser')
    
    list_pages = []
    
    pages = int(
        soup.find
        (
            'a',
            class_='toggle-menu__item-link toggle-menu__item-link_pagination toggle-menu__item-link_bordered'
        )
        .get('href')
        [-3:-1]
    )
                
    for page in range(1, pages +1):
    
        list_pages += 'https://habr.com/ru/all/page' + str(page) + '/',
    return list_pages 

In [3]:
def get_all_links():
    
    """
    Функция выводит список url:
    ['https://habr.com/ru/post/521646/'
     'https://habr.com/ru/post/516756/'
     ...]
     
    """  
    # определяем список ключевых слов которые нам интересны
    keywords = ['python', 'парсинг']
    
    all_links = []
    for url_page in get_pages():
        
        habr = requests.get(url_page)
        
        soup = BeautifulSoup(habr.text, 'html.parser')
        posts = soup.find_all('h2', class_='post__title')

        for post in posts:
            post_link = post.find('a', class_='post__title_link')
            link = post_link.get('href')

            title = post_link.get_text()

            for key in keywords: 
                if key in title:
                    all_links.append(link)                   
    
    return all_links

all_links = get_all_links()

In [4]:
def get_kom_news(links):

    kom_news = pd.DataFrame()

    for link in all_links:
        time.sleep(0.3)
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')

    #   Получаем даты 
        date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()

    #   Получаем названия статей 
        post_text = soup.find('span', class_='post__title-text')
        title = post_text.get_text()

    #   Собираем DataFrame
        row = {'date': date, 'title': title, 'link':link }

        kom_news = pd.concat([kom_news, pd.DataFrame([row])], ignore_index=True) 
        
    return kom_news
get_kom_news(all_links)

,date,title,link
0,2020-10-01,Как выбрать решение для парсинга сайтов: класс...,https://habr.com/ru/post/521646/
1,2020-10-01,"Как разложить фото, видео по папкам, исходя из...",https://habr.com/ru/post/516756/


### Задание 2

In [55]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
URL_avast = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
adresses = {'emailAddresses': EMAIL}

In [57]:
res = requests.post(URL_avast, data = json.dumps(adresses), headers = headers).json()
res

{'breaches': {'16613': {'site': 'verifications.io',
   'recordsCount': 677914246,
   'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n',
   'publishDate': '2019-03-28T00:00:00Z'},
  '17609': {'site': 'vk.com',
   'recordsCount': 42542545,
   'description': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and additional personal information. This breach is being privately shared on the internet.',
   'publishDate': '2020-0

In [59]:
df = pd.DataFrame(res['breaches']).transpose().reset_index() 
df_= df[['site', 'description', 'publishDate']]
df_

,site,description,publishDate
0,verifications.io,Big data e-mail verification platform verifica...,2019-03-28T00:00:00Z
1,vk.com,"At some time in 2020, the Russian social netwo...",2020-05-21T00:00:00Z
2,azcentral.com,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z
3,wishbone.io,"In January 2020, the online poll website Wishb...",2020-05-28T00:00:00Z
4,myheritage.com,"In October 2017, a customer database belonging...",2017-11-04T00:00:00Z
5,canva.com,"In May 2019, graphic-design site Canva's datab...",2019-06-13T00:00:00Z
6,parapa.mail.ru,"In July and August 2016, two criminals execute...",2017-02-14T00:00:00Z
7,dropbox.com,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z
8,linkedin.com,"In 2012, online professional networking platfo...",2016-10-21T00:00:00Z
9,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra...",2017-03-01T00:00:00Z
